In [8]:
import os

import pandas as pd

from const import *

genre_df = pd.read_csv(os.path.join(DATA_DIR, "genres.csv"), index_col=0)

In [12]:
# get genre count range for outlier detection
genre_counts = genre_df.groupby(ID).count()
count_mean = genre_counts[GENRE].mean()
count_std = genre_counts[GENRE].std()

count_range = (count_mean - 3 * count_std, count_mean + 3 * count_std)
count_range

(-0.9036903575613164, 4.00647881789528)

In [20]:
# number of outliers
filtered_movies = genre_counts[genre_counts[GENRE] < count_range[1]]
len(genre_counts.index) - len(filtered_movies.index)

3827

In [42]:
valid_movies = pd.read_csv("valid_movies.csv", index_col=0)
cleaned_movies = pd.Series(
    sorted(list(set(valid_movies[MOVIE_ID]).intersection(set(filtered_movies.index))))
)
cleaned_genre_df = genre_df[genre_df.index.isin(cleaned_movies)]
encoded_df = (
    pd.get_dummies(cleaned_genre_df, columns=[GENRE], prefix="").groupby(ID).any()
)
encoded_df["genre_count"] = encoded_df.sum(axis=1)
encoded_df.index.names = [MOVIE_ID]
encoded_df.to_csv("encoded_genres.csv")